In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc gem-suite matplotlib

*تقدير الاستخدام: 3 دقائق على معالج Heron r2 (ملاحظة: هذا تقدير فحسب. قد يختلف وقت التشغيل الفعلي لديك.)*

## الخلفية

يوضح هذا البرنامج التعليمي كيفية تحقيق انتقال طور نيشيموري على معالج كم IBM&reg;. وُصف هذا التجربة في الأصل في [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

يشير انتقال طور نيشيموري إلى الانتقال بين الأطوار ذات الترتيب قصير المدى وبعيد المدى في نموذج Ising ذي الروابط العشوائية. على الحاسوب الكمي، يتجلى الطور ذو الترتيب بعيد المدى على شكل حالة تكون فيها البتات الكمومية متشابكة عبر الجهاز بأكمله. تُحضَّر هذه الحالة شديدة التشابك باستخدام بروتوكول *توليد التشابك بالقياس* (GEM). وبفضل استخدام القياسات وسط الدارة، يستطيع بروتوكول GEM تشبيك البتات الكمومية عبر الجهاز بأكمله باستخدام دوائر ذات عمق ثابت فحسب. يستخدم هذا البرنامج التعليمي تنفيذ بروتوكول GEM من حزمة برامج [GEM Suite](https://github.com/qiskit-community/gem-suite).

## المتطلبات

قبل البدء في هذا البرنامج التعليمي، تأكد من تثبيت ما يلي:

- Qiskit SDK الإصدار 1.0 أو أحدث، مع دعم [التصور](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime الإصدار 0.22 أو أحدث ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## الإعداد

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## الخطوة 1: ربط المدخلات الكلاسيكية بمسألة كمية
يعمل بروتوكول GEM على معالج كمي تُوصف فيه ترابطية البتات الكمومية بشبكة. تستخدم معالجات IBM الكمية الحالية [شبكة heavy hex](https://www.ibm.com/quantum/blog/heavy-hex-lattice). تُجمَّع البتات الكمومية للمعالج في *لويحات* (plaquettes) بناءً على خلية الوحدة التي تشغلها في الشبكة. ونظرًا لإمكانية ظهور بتة كمومية واحدة في أكثر من خلية وحدة، فإن اللويحات ليست متقاطعة. في شبكة heavy hex، تحتوي اللويحة على 12 بتة كمومية. كما تُشكّل اللويحات بحد ذاتها شبكة، حيث تكون لويحتان متصلتين إذا كانتا تتشاركان أي بتات كمومية. في شبكة heavy hex، تتشارك اللويحات المتجاورة 3 بتات كمومية.

في حزمة برامج GEM Suite، الفئة الأساسية لتنفيذ بروتوكول GEM هي `PlaquetteLattice`، التي تمثل شبكة اللويحات (وهي مختلفة عن شبكة heavy hex). يمكن تهيئة `PlaquetteLattice` من خريطة اقتران البتات الكمومية. حاليًا، لا تُدعم إلا خرائط اقتران heavy hex.

تُهيئ خلية الكود التالية شبكة لويحات من خريطة اقتران معالج IBM كمي. لا تشمل شبكة اللويحات دائمًا الجهاز بأكمله. على سبيل المثال، يحتوي `ibm_torino` على 133 بتة كمومية إجمالًا، لكن أكبر شبكة لويحات تناسب الجهاز تستخدم 125 منها فحسب، وتضم 18 لويحة إجمالًا. يمكن ملاحظة ما يشبه ذلك في أجهزة IBM Quantum&reg; ذات أعداد مختلفة من البتات الكمومية أيضًا.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

يمكنك تصور شبكة اللويحات بإنشاء مخطط لتمثيلها البياني. في المخطط، تُمثَّل اللويحات بسداسيات مُعلَّمة، وتُربط لويحتان بحافة إذا كانتا تتشاركان بتات كمومية.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

يمكنك استرداد معلومات حول اللويحات الفردية، مثل البتات الكمومية التي تحتوي عليها، باستخدام طريقة `plaquettes`.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

يمكنك أيضًا إنتاج مخطط للبتات الكمومية الأساسية المكوِّنة لشبكة اللويحات.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

إضافة إلى تسميات البتات الكمومية والحواف الدالة على اتصالها، يتضمن المخطط ثلاث معلومات إضافية ذات صلة ببروتوكول GEM:
- كل بتة كمومية إما مظللة (باللون الرمادي) أو غير مظللة. البتات المظللة هي بتات "الموقع" التي تمثل مواقع نموذج Ising، أما البتات غير المظللة فهي بتات "الرابطة" المستخدمة للوساطة في التفاعلات بين بتات الموقع.
- كل بتة موقع مُعلَّمة بـ (A) أو (B)، مما يشير إلى أحد دورين يمكن أن تضطلع بهما بتة الموقع في بروتوكول GEM (يُشرح الدوران لاحقًا).
- كل حافة مُلوَّنة بإحدى ست ألوان، مما يقسم الحواف إلى ست مجموعات. يحدد هذا التقسيم كيفية توازي بوابات ثنائية البتات الكمومية، فضلًا عن أنماط جدولة مختلفة يُرجَّح أن تُسبب قدرًا مختلفًا من الأخطاء على معالج كمي مُشوَّش. ونظرًا لأن الحواف في كل مجموعة متفرقة، يمكن تطبيق طبقة من بوابات ثنائية البتات على تلك الحواف في آنٍ واحد. في الواقع، من الممكن تقسيم الألوان الست إلى ثلاث مجموعات من لونين بحيث يظل اتحاد كل مجموعة من اللونين متفرقًا. وبالتالي، لا تلزم سوى ثلاث طبقات من بوابات ثنائية البتات لتنشيط كل حافة. ثمة 12 طريقة لتقسيم الألوان الست على هذا النحو، وينتج عن كل تقسيم جدول بوابات مختلف من 3 طبقات.

الآن بعد أن أنشأت شبكة لويحات، تتمثل الخطوة التالية في تهيئة كائن `GemExperiment`، مع تمرير شبكة اللويحات والخلفية التي تعتزم تشغيل التجربة عليها. تدير فئة `GemExperiment` التنفيذ الفعلي لبروتوكول GEM، بما في ذلك توليد الدوائر وإرسال المهام وتحليل البيانات. تُهيئ خلية الكود التالية فئة التجربة مع تقييد شبكة اللويحات بلويحتين فحسب (21 بتة كمومية)، مما يُقلص حجم التجربة لضمان عدم طغيان الضوضاء في الجهاز على الإشارة.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

تُبنى دارة بروتوكول GEM وفق الخطوات التالية:
1. تحضير حالة $|+\rangle$ الكلية بتطبيق بوابة Hadamard على كل بتة كمومية.
2. تطبيق بوابة $R_{ZZ}$ بين كل زوج من البتات الكمومية المتصلة. يمكن تحقيق ذلك باستخدام 3 طبقات من البوابات. تعمل كل بوابة $R_{ZZ}$ على بتة موقع وبتة رابطة. إذا كانت بتة الموقع مُعلَّمة بـ (B)، فإن الزاوية تكون ثابتة عند $\frac{\pi}{2}$. أما إذا كانت بتة الموقع مُعلَّمة بـ (A)، فإن الزاوية مسموح لها بالتغيير، مما ينتج دوائر مختلفة. بشكل افتراضي، يُضبط نطاق الزوايا على 21 نقطة متساوية التباعد بين $0$ و$\frac{\pi}{2}$ شاملةً طرفيه.
3. قياس كل بتة رابطة في أساس Pauli $X$. نظرًا لأن البتات الكمومية تُقاس في أساس Pauli $Z$، يمكن تحقيق ذلك بتطبيق بوابة Hadamard قبل قياس البتة الكمومية.

لاحظ أن الورقة المشار إليها في مقدمة هذا البرنامج التعليمي تستخدم اصطلاحًا مختلفًا لزاوية $R_{ZZ}$، يختلف عن الاصطلاح المستخدم هنا بعامل مقداره 2.

في الخطوة 3، تُقاس بتات الرابطة فحسب. لفهم الحالة التي تبقى عليها بتات الموقع، من المفيد النظر في الحالة التي تكون فيها زاوية $R_{ZZ}$ المُطبَّقة على بتات الموقع (A) في الخطوة 2 مساوية لـ $\frac{\pi}{2}$. في هذه الحالة، تبقى بتات الموقع في حالة شديدة التشابك تشبه حالة GHZ،

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

نظرًا لعشوائية نتائج القياس، قد تكون الحالة الفعلية لبتات الموقع حالة مختلفة ذات ترتيب بعيد المدى، مثل $\lvert 00110 \rangle + \lvert 11001 \rangle$. غير أن حالة GHZ يمكن استعادتها بتطبيق عملية فك ترميز مبنية على نتائج القياس. عند خفض زاوية $R_{ZZ}$ من $\frac{\pi}{2}$، لا يزال بالإمكان استعادة الترتيب بعيد المدى حتى زاوية حرجة تبلغ في غياب الضوضاء نحو $0.3 \pi$. دون هذه الزاوية، لا تُظهر الحالة الناتجة تشابكًا بعيد المدى بعد الآن. هذا الانتقال بين وجود الترتيب بعيد المدى وغيابه هو انتقال طور نيشيموري.

في الوصف أعلاه، بقيت بتات الموقع دون قياس، ويمكن إجراء عملية فك الترميز بتطبيق بوابات كمية. أما في التجربة كما نُفِّذت في GEM suite، والتي يتبعها هذا البرنامج التعليمي، فإن بتات الموقع تُقاس فعلًا، وتُطبَّق عملية فك الترميز في خطوة معالجة كلاسيكية لاحقة.

في الوصف أعلاه، يمكن إجراء عملية فك الترميز بتطبيق بوابات كمية على بتات الموقع لاستعادة الحالة الكمومية. أما إذا كان الهدف هو قياس الحالة فورًا، مثلًا لأغراض التوصيف، فإن بتات الموقع تُقاس مع بتات الرابطة، ويمكن تطبيق عملية فك الترميز في خطوة معالجة كلاسيكية لاحقة. وهكذا تُنفَّذ التجربة في GEM suite، والتي يتبعها هذا البرنامج التعليمي.

إضافة إلى اعتمادها على زاوية $R_{ZZ}$ في الخطوة 2 التي تجتاح 21 قيمة بشكل افتراضي، تعتمد دارة بروتوكول GEM أيضًا على نمط الجدولة المستخدم لتنفيذ 3 طبقات من بوابات $R_{ZZ}$. كما ناقشنا سابقًا، ثمة 12 نمط جدولة من هذا القبيل. وبالتالي، يبلغ إجمالي عدد الدوائر في التجربة $21 \times 12 = 252$.

يمكن توليد دوائر التجربة باستخدام طريقة `circuits` من فئة `GemExperiment`.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

لأغراض هذا البرنامج التعليمي، يكفي النظر في نمط جدولة واحد فحسب. تُقيِّد خلية الكود التالية التجربة بنمط الجدولة الأول. ونتيجة لذلك، تحتوي التجربة على 21 دارة فحسب، واحدة لكل زاوية $R_{ZZ}$ يُجتاح فوقها.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

ترسم خلية الكود التالية مخططًا للدارة ذات الفهرس 5. لتقليل حجم المخطط، تُزال بوابات القياس في نهاية الدارة.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## الخطوة 2: تحسين المسألة لتنفيذها على العتاد الكمي
يتضمن تحويل الدوائر الكمية للتنفيذ على العتاد عادةً [عدة مراحل](/guides/transpiler-stages). في الغالب، أكثر المراحل تكلفةً من الناحية الحسابية هي اختيار تخطيط البتات الكمومية، وتوجيه بوابات ثنائية البتات لتتوافق مع ترابطية البتات في العتاد، وتحسين الدارة لتقليل عدد البوابات وعمقها. في بروتوكول GEM، مرحلتا التخطيط والتوجيه غير ضروريتين لأن ترابطية العتاد مدمجة بالفعل في تصميم البروتوكول. الدوائر تحتوي مسبقًا على تخطيط للبتات الكمومية، وبوابات ثنائية البتات مُعيَّنة بالفعل على الاتصالات الأصلية. علاوة على ذلك، وللحفاظ على بنية الدارة عند تغيير زاوية $R_{ZZ}$، ينبغي إجراء تحسين أساسي فحسب للدارة.

تُجري فئة `GemExperiment` تحويل الدوائر بشفافية عند تنفيذ التجربة. مرحلتا التخطيط والتوجيه مُعَوَّضتان بالفعل افتراضيًا لتقوما بلا شيء، ويُجرى تحسين الدارة بمستوى يُحسِّن بوابات البتات الفردية فحسب. غير أنه يمكنك تجاوز هذه الإعدادات أو تمرير خيارات إضافية باستخدام طريقة `set_transpile_options`. للتوضيح، تُحوِّل خلية الكود التالية الدارة المعروضة سابقًا يدويًا وترسم الدارة المُحوَّلة.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## الخطوة 3: التنفيذ باستخدام Qiskit primitives
لتنفيذ دوائر بروتوكول GEM على العتاد، استدعِ طريقة `run` من كائن `GemExperiment`. يمكنك تحديد عدد اللقطات التي تريد أخذ عينات منها من كل دارة. تُعيد طريقة `run` كائنًا من نوع [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) ينبغي حفظه في متغير. لاحظ أن طريقة `run` تُرسل المهام فحسب دون انتظار اكتمالها، وبالتالي فهي استدعاء غير محجوب.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

لانتظار النتائج، استدعِ طريقة `block_for_results` من كائن `ExperimentData`. سيتسبب هذا الاستدعاء في تعليق المفسر حتى تنتهي المهام.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## الخطوة 4: المعالجة اللاحقة وإعادة النتيجة بالتنسيق الكلاسيكي المطلوب
عند زاوية $R_{ZZ}$ تبلغ $\frac{\pi}{2}$، ستكون الحالة المُفكَّك ترميزها هي حالة GHZ في غياب الضوضاء. يمكن تصور الترتيب بعيد المدى لحالة GHZ برسم مغنطة سلاسل البتات المقيسة. تُعرَّف المغنطة $M$ بوصفها مجموع معاملات Pauli $Z$ أحادية البتة،
$$
M = \sum_{j=1}^N Z_j,
$$
حيث $N$ هو عدد بتات الموقع. قيمتها لسلسلة بتات تساوي الفرق بين عدد الأصفار وعدد الآحاد. يعطي قياس حالة GHZ حالة الأصفار الكلية أو حالة الآحاد الكلية باحتمال متساوٍ، لذا ستكون المغنطة $+N$ في نصف الأحيان و$-N$ في النصف الآخر. في وجود الأخطاء الناجمة عن الضوضاء، ستظهر قيم أخرى أيضًا، لكن إذا لم تكن الضوضاء كبيرة جدًا، سيظل التوزيع مرتكزًا بالقرب من $+N$ و$-N$.

بالنسبة لسلاسل البتات الخام قبل فك الترميز، سيكون توزيع المغنطة مكافئًا لتوزيع سلاسل البتات العشوائية الموزعة توزيعًا منتظمًا في غياب الضوضاء.

ترسم خلية الكود التالية مغنطة سلاسل البتات الخام والمُفكَّك ترميزها عند زاوية $R_{ZZ}$ تبلغ $\frac{\pi}{2}$.